# Aнализ взаимодействия пользователей с карточками Яндекс.Дзен

**Цель исследования** - используя дашборд, ответить на следующие вопросы менеджеров:

* Cколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
* Как много карточек генерируют источники с разными темами?
* Как соотносятся темы карточек и темы источников?

Сам дашборд необходимо создать  на основе следующего ТЗ:

**Насколько часто предполагается пользоваться дашбордом:** не реже, чем раз в неделю;


**Кто будет основным пользователем дашборда:** менеджеры по анализу контента;


**Состав данных для дашборда:**
* История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
* Разбивка событий по темам источников;
* Таблица соответствия тем источников темам карточек;


**По каким параметрам данные должны группироваться:**
* Дата и время;
* Тема карточки;
* Тема источника;
* Возрастная группа;



**Характер данных:**
* История событий по темам карточек — абсолютные величины с разбивкой по минутам;
* Разбивка событий по темам источников — относительные величины (% событий);
* Соответствия тем источников темам карточек - абсолютные величины;


**Важность:** все графики имеют равную важность;


**Источники данных для дашборда:** cырые данные о событиях взаимодействия пользователей с карточками (таблица log_raw);


**База данных, в которой будут храниться агрегированные данные:** дополнительные агрегированные таблицы в БД zen;


**Частота обновления данных:** один раз в сутки, в полночь по UTC;


**Mакет дашборда:**

![](https://pictures.s3.yandex.net/resources/Untitled_-_2020-07-06T161108.832_1594041107.png)

**Ход исследования:**

 Исследование пройдёт в три этапа:
 1. Подключение к БД и подготовка данных для анализа
 2. Создание дашборда
 3. Анализ данных и формулирование выводов 
 4. Подготовка презентации с отчётом

# Подключение к БД и подготовка данных для анализа.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#подключаемся к базе

db_config = {'user': '*****', # имя пользователя
            'pwd': '*****', # пароль
            'host': '*****',
            'port': *****, # порт подключения
            'db': '*****'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string) 

query = '''
        SELECT *
        FROM dash_visits
'''

# выполняем sql запрос и 
data_raw = pd.io.sql.read_sql(query, con = engine)



Предварительно ознакомимся с данными

In [3]:
data_raw.head()

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


In [4]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    30745 non-null  int64 
 1   record_id     30745 non-null  int64 
 2   item_topic    30745 non-null  object
 3   source_topic  30745 non-null  object
 4   age_segment   30745 non-null  object
 5   dt            30745 non-null  object
 6   visits        30745 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 1.6+ MB


В данных отсутствуют пропуски. Типы данных корректны для всех столбцов, кроме `dt` - здесь требуется перевести значения в`datetime`

Дополнительно проверим столбцы на наличие аномалий

In [7]:
data_raw.age_segment.value_counts()

18-25    7056
26-30    5875
31-35    5552
36-40    5105
41-45    3903
45+      3254
Name: age_segment, dtype: int64

In [8]:
data_raw.source_topic.value_counts()

Семейные отношения    1822
Россия                1687
Знаменитости          1650
Полезные советы       1578
Путешествия           1563
Кино                  1505
Дети                  1459
История               1437
Семья                 1405
Одежда                1379
Здоровье              1243
Искусство             1228
Авто                  1077
Психология            1055
Сад и дача            1036
Политика              1024
Спорт                 1007
Сделай сам             995
Ремонт                 985
Деньги                 973
Еда                    912
Интерьеры              809
Строительство          758
Музыка                 750
Технологии             741
Финансы                667
Name: source_topic, dtype: int64

In [9]:
data_raw.item_topic.value_counts()

Отношения             1536
Интересные факты      1535
Наука                 1505
Подборки              1456
Полезные советы       1424
Общество              1422
Россия                1385
История               1363
Семья                 1287
Путешествия           1247
Деньги                1234
Женщины               1230
Дети                  1229
Туризм                1206
Здоровье              1203
Красота               1193
Культура              1160
Юмор                  1129
Искусство             1119
Рассказы              1109
Психология            1056
Скандалы              1023
Знаменитости           976
Женская психология     914
Шоу                    804
Name: item_topic, dtype: int64

In [10]:
 data_raw.visits.describe() 

count    30745.000000
mean        10.089673
std         19.727601
min          1.000000
25%          1.000000
50%          3.000000
75%         10.000000
max        371.000000
Name: visits, dtype: float64

Ни в одном из столбцов не замечено каких-либо аномалий в данных. Также проверим датафрейм на явные дубликаты

In [11]:
data_raw.duplicated().sum()

0

Дубликаты также отсутствуют. Осталось только изменить тип данных в столбце `dt`, и датафрейм будет полностью готов для построения дашборда

In [12]:
data_raw['dt'] = pd.to_datetime(data_raw['dt'])

In [14]:
# проверка результата
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


In [16]:
# сохраняем обработанные данные в csv файл
#data_raw.to_csv('dash_visits.csv')

# Создание дашборда

Ссылка на дашборд https://public.tableau.com/views/zen_dash_16620319522300/Dashboard1?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link

# Анализ данных и формулирование выводов

## Cколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?

Всего за указанный период пользователи совершили чуть более 310 тыс. взаимодействий с карточками. 

Отчетливо выделяются 2 пика пользовательской активности:
* малый - в районе 18:30;
* и большой - в районе 19:00

B остальное время пользователи активно не взаимодействовали с сервисом. 



* Самые популярные темы карточек среди **всех пользователей** (в порядке убывания пользовательского интереса) - **Наука, Отношения, Интересные факты, Общество, Подборки**.


* Топ-5 среди возрастной категории **18-25** - **Отношения, Наука, Общество, Интересные факты, Россия**


* Топ-5 среди возрастной категории **26-30** - **Наука, Интересные факты, Общество, Отношения, Подборки**


* Топ-5 среди возрастной категории **31-35** - **Наука, Интересные факты, Общество, Подборки, Отношения**


* Топ-5 среди возрастной категории **36-40** - **Наука, Интересные факты, Подборки, Отношения, Общество**


* Топ-5 среди возрастной категории **41-45** - **Наука, Отношения, Интересные факты, Общество, Подборки**


* Топ-5 среди возрастной категории **45+** - **Наука, Интересные факты, Общество, Отношения, Подборки**

**Выводы**

* Безоговорочным лидером почти среди всех возрастных категорий являются карточки с темой "Наука"

* Лишь для самой молодой аудитории наиболее интересной темой являются "Отношения"

* Помимо этого, пользователи категории 18-25 единственные, у которых в число наиболее интересных тем попали карточки, посвященные России. 

## Как много карточек генерируют источники с разными темами?

Самые популярные темы карточек среди всех пользователей (в порядке убывания пользовательского интереса) - **Семейные отношения, Россия, Полезные советы, Путешествия, Знаменитости, Кино**. На них приходится более половины от всех взаимодействий пользователей. 


* Топ-6 среди возрастной категории **18-25** - **Россия, Семейные отношения, Полезные советы, Путешествия, Знаменитости, Кино**


* Топ-6 среди возрастной категории **26-30** - **Семейные отношения, Россия, Полезные советы, Путешествия, Знаменитости, Кино**


* Топ-6 среди возрастной категории **31-35** - **Семейные отношения, Знаменитости,  Россия, Путешествия, Полезные советы, Кино**


* Топ-6 среди возрастной категории **36-40** - **Семейные отношения, Знаменитости,  Россия, Полезные советы, Кино, Путешествия**


* Топ-6 среди возрастной категории **41-45** - **Семейные отношения, Россия, Знаменитости, Кино, Полезные советы, Путешествия**


* Топ-6 среди возрастной категории **45+** - **Семейные отношения, Россия, Полезные советы, Знаменитости, Путешествия, Кино**

**Выводы**

* Для всех пользователей, за исключением самых молодых, источники про Семейные отношения являются самыми популярными. Молодежь же предпочитает тематику, связанную с Россией. 

* Также можно заметить, что с увеличением возраста несколько возрастает интерес к источникам про  Знаменитостей. 

## Как соотносятся темы карточек и темы источников?

Самые популярные комбинации тем карточек - тем источников среди всех пользователей (в формате "Карточка-Источник"): **Рассказы-Путешествия, Общество-Россия, Наука-Кино, Россия-Россия, Подборки-Полезные советы**


* Топ-5 среди возрастной категории **18-25** - **Рассказы-Путешествия, Общество-Россия, Отношения-Полезные советы, Россия-Россия, Наука-Кино**


* Топ-5 среди возрастной категории **26-30** - **Рассказы-Путешествия, Общество-Россия, Наука-Кино, Интересные факты-Россия, Россия-Россия**


* Топ-5 среди возрастной категории **31-35** - **Рассказы-Путешествия, Наука-Кино,  Общество-Россия, Общество-Семейные отношения, Россия-Россия**


* Топ-5 среди возрастной категории **36-40** - **Рассказы-Путешествия, Наука-Кино, Общество-Семейные отношения, Общество-Россия, Подборки-Полезные советы**


* Топ-5 среди возрастной категории **41-45** - **Рассказы-Путешествия, Наука-Кино, Женщины-Семейные отношения, Общество-Семейные отношения, История-Путешествия**


* Топ-5 среди возрастной категории **45+** - **Рассказы-Путешествия, Общество-Россия, Общество-Семейные отношения, Наука-Кино, Подборки-Полезные советы**

**Выводы**

* Чаще всего пользователи заходят на Дзен, чтобы почитать чьи-нибудь рассказы о совершенных авторами путешествиях. Подобные карточки абсолютные лидеры по всех возрастным категориям. 

* С увеличением возраста, пользователи все большее внимание уделяют карточкам, с источниками про Семейные отношения. 

* Пользователи среднего возраста, по сравнению с другими, проявляют меньший интерес к карточкам, связанным с обществом и темой источника с тематикой "Россия". 

# Подготовка презентации с отчётом

файл с презентацией лежит в архиве